In [1]:
pip install requests d6tflow psycopg2 boto3

Note: you may need to restart the kernel to use updated packages.


In [2]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import d6tflow 
import requests 
import pandas as pd 
import xml.etree.ElementTree as ET

# Get the current date and time
now = datetime.now()
# Subtract two years
two_years_ago = now - relativedelta(years=2)
print(two_years_ago)

Welcome to d6tflow! For Q&A see https://github.com/d6t/d6tflow
2022-09-24 17:42:00.717454


In [3]:
query_words = ['machine', 'learning', 'meta', 'data']
queries = [word + '&' for word in query_words]
query = ''.join(queries)
query_size = 3000
url = f'https://export.arxiv.org/api/query?search_query=all:{query}start=0&max_results={query_size}'
print(url)
word = query_words[:2]
word = ' '.join(word)
print(word)

https://export.arxiv.org/api/query?search_query=all:machine&learning&meta&data&start=0&max_results=3000
machine learning


In [4]:
data = requests.get(url).text

In [5]:
class MetaDataTask(d6tflow.tasks.TaskPickle):
    def run(self):
        archive = dict()
        archive['url'] = url 
        archive["data"] = data
        df = {'meta': archive}
        self.save(df)
        
flow = d6tflow.Workflow()
flow.run(MetaDataTask)


===== Luigi Execution Summary =====

Scheduled 1 tasks of which:
* 1 complete ones were encountered:
    - 1 MetaDataTask()

Did not run any tasks
This progress looks :) because there were no failed tasks or missing dependencies

===== Luigi Execution Summary =====



LuigiRunResult(status=<LuigiStatusCode.SUCCESS: (':)', 'there were no failed tasks or missing dependencies')>,worker=<luigi.worker.Worker object at 0x112c417d0>,scheduling_succeeded=True)

In [12]:
class SortMetaTask(d6tflow.tasks.TaskPickle):
  
    def run(self):
        two_years_ago = pd.Timestamp.now(tz='UTC') - pd.DateOffset(years=2)
        # df = pd.read_xml(data)
        from io import StringIO
        df = pd.read_xml(StringIO(data) if isinstance(data, str) else data)
        index = pd.RangeIndex(start=0, stop=3000)
        xml_df = pd.DataFrame()
        xml_df['title'] = df['title'][7:]
        xml_df['abstract'] = df['summary'][7:]
        xml_df['published'] = df['published'][7:]
        xml_df['published'] = pd.to_datetime(df['published'])
        xml_df['updated'] = df['updated'][7:]
        xml_df['url'] = df['id'][7:]
        xml_df['two_year_date'] = xml_df['published'].apply(lambda x: 1 if x > two_years_ago else 0)
        xml_df['title_has_word'] = xml_df['title'].str.contains(f'{word}', case=False)
        xml_df['combined'] = xml_df['title'] + ' ' + xml_df['abstract']
        # xml_df.index = index
        self.save(xml_df)
        
# flow = d6tflow.Workflow()
# flow.run(SortMetaTask)

In [13]:
flow = d6tflow.Workflow()
flow.run(SortMetaTask)


===== Luigi Execution Summary =====

Scheduled 1 tasks of which:
* 1 ran successfully:
    - 1 SortMetaTask()

This progress looks :) because there were no failed tasks or missing dependencies

===== Luigi Execution Summary =====



LuigiRunResult(status=<LuigiStatusCode.SUCCESS: (':)', 'there were no failed tasks or missing dependencies')>,worker=<luigi.worker.Worker object at 0x125faf150>,scheduling_succeeded=True)